For each 24 hour day (midnight tomidnight) we'd like to know the following:
* Temperature and barometric pressure values at sunrise and sunset
* Difference between temp and pressure at sunrise and sunset
* Maximum, minimum and averages for temp and pressure for both of the time frames of sunset to sunrise and sunrise to sunset
* Differences between the max, min and average for both of these time frames.


# Load libraries

In [473]:
# pandas for data structure
import pandas as pd
import datetime

# Load data

#### Weather Data

* data collected from [Wunderground](https://www.wunderground.com/weather/api/)
* hosted file: [Google Drive](https://drive.google.com/file/d/1eS0gGM14g7iFulUeqz3XwbKb5OtK9aSI/view)

In [474]:
# local file
filename_wunderground = '../data/wunderground-170701_171101-day_night.csv'

In [475]:
# load data into dataframes
wund = pd.read_csv(filename_wunderground, parse_dates=['utc_date'])

In [476]:
wund['utc_date'] = wund['utc_date'].dt.tz_localize('utc')

In [477]:
# localize datetime make local_date column
wund['local_date'] = pd.to_datetime(wund.loc[:, 'utc_date']).dt.tz_convert('US/Mountain')

In [478]:
wund = wund.set_index('local_date')

In [479]:
wund = wund[['station_id', 'pressurei', 'pressurem', 'tempi', 'tempm', 'utc_date']]

In [480]:
wund.columns = ['station_id', 'pressi', 'pressm', 'tempi', 'tempm', 'utc_date']

In [481]:
wund.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 727764 entries, 2017-07-01 00:12:00-06:00 to 2017-10-31 23:56:00-06:00
Data columns (total 6 columns):
station_id    727764 non-null object
pressi        727764 non-null float64
pressm        727764 non-null float64
tempi         727764 non-null float64
tempm         727764 non-null float64
utc_date      727764 non-null datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(4), object(1)
memory usage: 38.9+ MB


In [483]:
wund.tail(2)

,station_id,pressi,pressm,tempi,tempm,utc_date
local_date,,,,,,
2017-10-31 23:51:00-06:00,KMTVICTO9,29.71,1006.0,50.9,10.5,2017-11-01 05:51:00+00:00
2017-10-31 23:56:00-06:00,KMTVICTO9,29.71,1006.0,50.5,10.3,2017-11-01 05:56:00+00:00


#### Sunset Sunrise Data

In [210]:
# Load Sunset Sunrise data
sun_filename = '../data/sunrise_sunset-wunderground-utc.csv'
sun = pd.read_csv(sun_filename, parse_dates=['sunrise', 'sunset'])

In [211]:
# Select a subset of loaded DataFrame
sun = sun[['station_id', 'sunrise', 'sunset']]

In [212]:
# Rename columns
sun.columns = ['station_id', 'sunrise_utc', 'sunset_utc']

In [213]:
# Localize datetime to UTC
sun['sunrise_utc'] = sun['sunrise_utc'].dt.tz_localize('utc')
sun['sunset_utc'] = sun['sunset_utc'].dt.tz_localize('utc')

In [214]:
# Create US/Mountain datetimes
sun['sunrise_local'] = pd.to_datetime(sun.loc[:, 'sunrise_utc']).dt.tz_convert('US/Mountain')
sun['sunset_local'] = pd.to_datetime(sun.loc[:, 'sunset_utc']).dt.tz_convert('US/Mountain')

In [246]:
# Reorder columns
sun = sun[['station_id', 'sunrise_local', 'sunset_local', 'sunrise_utc', 'sunset_utc']]

In [250]:
# Drop rows which start 2017-06-30
sun = sun[sun.sunset_local.dt.month != 6]

In [254]:
# Reset index
sun = sun.reset_index(drop=True)

In [259]:
sun.head(2)

,station_id,sunrise_local,sunset_local,sunrise_utc,sunset_utc
0,KMTCORVA9,2017-07-01 05:48:49-06:00,2017-07-01 21:31:41-06:00,2017-07-01 11:48:49+00:00,2017-07-02 03:31:41+00:00
1,KMTCORVA9,2017-07-02 05:49:27-06:00,2017-07-02 21:31:25-06:00,2017-07-02 11:49:27+00:00,2017-07-03 03:31:25+00:00


# New DataFrame

In [453]:
# Column names
columns = ['date', 'station_id', 'rise_tempi','set_tempi','rise_pressi','set_pressi']

In [454]:
calcs = pd.DataFrame(columns=columns)

#### Populate with stations and date range

In [455]:
# Create date range by day
start = datetime.date(2017, 7, 1)
end = datetime.date(2017, 10, 31)
date_index = pd.date_range(start, periods=123, freq='D')

In [456]:
# Append days and unique station ids
for station in sun.station_id.unique():
    for date in date_index:
        calcs = calcs.append({'station_id': station, 'date': date.strftime('%Y-%m-%d')},
                            ignore_index=True)

In [459]:
calcs = calcs.set_index('date')

In [461]:
calcs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4059 entries, 2017-07-01 to 2017-10-31
Data columns (total 5 columns):
station_id     4059 non-null object
rise_tempi     0 non-null object
set_tempi      0 non-null object
rise_pressi    0 non-null object
set_pressi     0 non-null object
dtypes: object(5)
memory usage: 190.3+ KB


# Calculations

### Column for values at Sunrise and Sunset
* 'values' refers to temperature and pressure data
* indexed by day
* date will go sunrise to sunset
* columns = ['rise_tempi','set_tempi','rise_pressurei','set_pressurei']
* index = ['2017-07-01', .... '2017-10-31']

* Get closest wund.local_date to sun.sunrise_local
* [query the closest datetime index](https://stackoverflow.com/questions/42264848/pandas-dataframe-how-to-query-the-closest-datetime-index)

In [525]:
def find_closest_weather_sample(station, date):
    wund_station = wund[wund.station_id == station].copy()
    idx = wund_station.index.get_loc(date, method='nearest')
    print(station, date)
    return wund_station.iloc[idx]

### Sunrise/Sunset Temp/Pressure

In [468]:
wund.head(1)

,station_id,pressi,pressm,tempi,tempm,utc_date
local_date,,,,,,
2017-07-01 00:12:00-06:00,KMTCORVA9,26.0,880.4,58.8,14.9,2017-07-01 06:12:00+00:00


In [462]:
calcs.head(1)

,station_id,rise_tempi,set_tempi,rise_pressi,set_pressi
date,,,,,
2017-07-01,KMTCORVA9,NaN,NaN,NaN,NaN


In [487]:
wund.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 727764 entries, 2017-07-01 00:12:00-06:00 to 2017-10-31 23:56:00-06:00
Data columns (total 6 columns):
station_id    727764 non-null object
pressi        727764 non-null float64
pressm        727764 non-null float64
tempi         727764 non-null float64
tempm         727764 non-null float64
utc_date      727764 non-null datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(4), object(1)
memory usage: 38.9+ MB


In [484]:
# Find weather samples for sunset and sunrise
for row in sun.tail(2).iterrows():
    station_id = row[1]['station_id']
    sunrise_local = row[1]['sunrise_local']
    sunset_local = row[1]['sunset_local']
    idx = find_closest_weather_sample(station_id, sunrise_local)
    wund_sample = wund.iloc[find_closest_weather_sample(station_id, sunrise_local)]
    wund_rise_temp = wund_sample.tempi
    wund_rise_press = wund_sample.pressi
    
    print('idx:', idx)
    print(sunrise_local.strftime('%Y-%m-%d'))
    print('**********************************************')
    print('*', station_id)
    print('Sunrise:', sunrise_local, '\n')
    
    print('* Nearest Weather Sample Datetime:')
    print(wund_sample)
    print('rise_temp :', wund_rise_temp)
    print('rise_press:', wund_rise_press)
    
    print('**********************************************')
    print('\n')

idx: 32685
2017-10-30
**********************************************
* KMTVICTO9
Sunrise: 2017-10-30 08:16:52-06:00 

* Nearest Weather Sample Datetime:
station_id                   KMTCORVA10
pressi                            30.15
pressm                           1020.9
tempi                              55.4
tempm                                13
utc_date      2017-09-16 21:32:00+00:00
Name: 2017-09-16 15:32:00-06:00, dtype: object
rise_temp : 55.4
rise_press: 30.15
**********************************************


idx: 32949
2017-10-31
**********************************************
* KMTVICTO9
Sunrise: 2017-10-31 08:18:19-06:00 

* Nearest Weather Sample Datetime:
station_id                   KMTCORVA10
pressi                            29.93
pressm                           1013.4
tempi                              64.8
tempm                              18.2
utc_date      2017-09-17 19:52:00+00:00
Name: 2017-09-17 13:52:00-06:00, dtype: object
rise_temp : 64.8
rise_press: 29.93
*

### Difference sunset, sunrise values

### Difference sunrise, sunset values

### Max, min, ave : sunset to sunrise : sunrise to sunset

### Difference max, min, ave sunset to sunrise : sunrise to sunset

### Scratch

In [335]:
start = datetime.date(2017, 7, 1)
end = datetime.date(2017, 10, 31)

In [334]:
start

datetime.date(2017, 7, 1)

In [345]:
date_index = pd.date_range(start, periods=123, freq='D')

In [348]:
test = pd.DataFrame(index=index, columns=columns)

In [346]:
index

DatetimeIndex(['2017-07-01', '2017-07-02', '2017-07-03', '2017-07-04',
               '2017-07-05', '2017-07-06', '2017-07-07', '2017-07-08',
               '2017-07-09', '2017-07-10',
               ...
               '2017-10-22', '2017-10-23', '2017-10-24', '2017-10-25',
               '2017-10-26', '2017-10-27', '2017-10-28', '2017-10-29',
               '2017-10-30', '2017-10-31'],
              dtype='datetime64[ns]', length=123, freq='D')

In [347]:
columns = ['x','y','z']

In [350]:
test.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 123 entries, 2017-07-01 to 2017-10-31
Freq: D
Data columns (total 3 columns):
x    0 non-null object
y    0 non-null object
z    0 non-null object
dtypes: object(3)
memory usage: 3.8+ KB
